In [1]:
#!pip install pandas numpy matplotlib xgboost scikit-learn tqdm

# Links
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html
* https://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics

In [1]:
import pandas as pd

dataset_name = 'original'
output_path = '.'
# test_path = '/kaggle/input/dataset/test.csv'
# train_path = '/kaggle/input/dataset/train.csv'
test_path = f'{dataset_name}_test.csv'
train_path = f'{dataset_name}_train.csv'
test_csv = pd.read_csv(test_path)
train_csv = pd.read_csv(train_path)
# sample_submission = pd.read_csv('sample_submission.csv')
train_csv


,id,price,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,...,weight_distribution_x,weight_distribution_y,engine_thrust,radar,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality
0,1,1845000.0,2122-11-28,3,1.0,1,59,2106/08,NaN,364.6,...,-0.070052,0.016326,1899.6,0,1,0,0,0,4,8
1,2,861600.0,2122-11-28,3,1.0,3,24,2115/08,NaN,998.3,...,0.104948,0.105376,1550.3,0,1,1,0,0,4,7
2,3,1446000.0,2122-11-28,4,2.0,2,24,2114/12,NaN,1793.4,...,0.110948,0.092526,2692.4,0,0,0,0,0,3,9
3,4,2250000.0,2122-11-28,3,1.0,2,51,2108/01,NaN,327.5,...,-0.082052,-0.005674,1496.6,0,1,1,0,2,4,8
4,5,1200000.0,2122-11-28,4,1.5,2,41,2112/11,NaN,2144.2,...,0.026948,0.149176,2507.2,0,1,1,0,0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,15125,1005000.0,2123-12-10,3,1.0,2,14,2114/01,NaN,2243.3,...,-0.063052,0.116376,1425.4,0,0,1,0,0,4,7
15125,15126,1080000.0,2123-12-10,4,2.0,2,20,2115/11,NaN,364.5,...,0.036448,0.079526,1150.8,0,1,1,0,0,3,7
15126,15127,2565000.0,2123-12-10,3,1.0,1,43,2110/01,NaN,200.0,...,-0.043552,-0.062674,1919.0,0,0,0,0,0,3,7
15127,15128,1336500.0,2123-12-20,2,1.0,1,50,2105/05,NaN,333.5,...,-0.032052,-0.065674,771.2,0,0,0,0,0,3,6


In [2]:
results = pd.DataFrame(columns=['dataset', 'method', 'variant', 'rmse', 'mse', 'mae'])
results

,dataset,method,variant,rmse,mse,mae


In [3]:
train_csv

,id,price,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,...,weight_distribution_x,weight_distribution_y,engine_thrust,radar,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality
0,1,1845000.0,2122-11-28,3,1.0,1,59,2106/08,NaN,364.6,...,-0.070052,0.016326,1899.6,0,1,0,0,0,4,8
1,2,861600.0,2122-11-28,3,1.0,3,24,2115/08,NaN,998.3,...,0.104948,0.105376,1550.3,0,1,1,0,0,4,7
2,3,1446000.0,2122-11-28,4,2.0,2,24,2114/12,NaN,1793.4,...,0.110948,0.092526,2692.4,0,0,0,0,0,3,9
3,4,2250000.0,2122-11-28,3,1.0,2,51,2108/01,NaN,327.5,...,-0.082052,-0.005674,1496.6,0,1,1,0,2,4,8
4,5,1200000.0,2122-11-28,4,1.5,2,41,2112/11,NaN,2144.2,...,0.026948,0.149176,2507.2,0,1,1,0,0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,15125,1005000.0,2123-12-10,3,1.0,2,14,2114/01,NaN,2243.3,...,-0.063052,0.116376,1425.4,0,0,1,0,0,4,7
15125,15126,1080000.0,2123-12-10,4,2.0,2,20,2115/11,NaN,364.5,...,0.036448,0.079526,1150.8,0,1,1,0,0,3,7
15126,15127,2565000.0,2123-12-10,3,1.0,1,43,2110/01,NaN,200.0,...,-0.043552,-0.062674,1919.0,0,0,0,0,0,3,7
15127,15128,1336500.0,2123-12-20,2,1.0,1,50,2105/05,NaN,333.5,...,-0.032052,-0.065674,771.2,0,0,0,0,0,3,6


In [4]:
train_csv.columns

Index(['id', 'price', 'sale_date', 'cabins', 'decks', 'bathrooms', 'producer',
       'created_date', 'refitted_date', 'total_area', 'crew_area',
       'cabins_area', 'maintenance_area', 'crew_area_coef', 'total_area_coef',
       'weight_distribution_x', 'weight_distribution_y', 'engine_thrust',
       'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality',
       'state', 'overall_quality'],
      dtype='object')

In [5]:
test_csv

,id,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,crew_area,...,weight_distribution_x,weight_distribution_y,engine_thrust,radar,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality
0,15130,2122-11-28,4,2.0,3,41,2116/06,NaN,361.6,155.5,...,0.026448,0.116076,1324.3,0,1,1,0,0,3,8
1,15131,2122-11-28,3,1.0,2,3,2114/07,NaN,541.7,104.5,...,-0.029552,0.103176,2408.2,0,0,0,0,0,4,8
2,15132,2122-11-28,4,2.0,2,37,2117/11,NaN,167.2,143.0,...,0.042948,-0.106724,1111.5,0,1,1,0,0,3,8
3,15133,2122-11-28,3,1.0,2,52,2107/03,NaN,237.5,130.0,...,-0.078052,-0.061624,1291.7,0,0,0,0,0,4,9
4,15134,2122-11-28,4,2.0,2,29,2117/03,NaN,2149.9,165.5,...,0.056448,-0.081574,2965.0,0,0,1,0,0,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6479,21609,2123-12-09,4,2.0,3,36,2117/09,NaN,902.1,124.5,...,-0.098552,0.071976,1528.8,0,1,1,0,0,3,8
6480,21610,2123-12-10,4,1.0,1,51,2109/02,NaN,292.5,99.0,...,-0.073552,-0.000324,1597.0,0,1,0,0,1,4,8
6481,21611,2123-12-10,5,2.0,3,43,2117/11,NaN,267.7,166.0,...,-0.043552,-0.047074,2173.0,0,1,1,0,0,3,9
6482,21612,2123-12-10,4,2.0,2,24,2117/09,NaN,589.0,129.0,...,0.101948,0.097126,2963.6,0,0,0,0,0,3,9


In [6]:
all_features = list(test_csv.columns)
test_csv.columns

Index(['id', 'sale_date', 'cabins', 'decks', 'bathrooms', 'producer',
       'created_date', 'refitted_date', 'total_area', 'crew_area',
       'cabins_area', 'maintenance_area', 'crew_area_coef', 'total_area_coef',
       'weight_distribution_x', 'weight_distribution_y', 'engine_thrust',
       'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality',
       'state', 'overall_quality'],
      dtype='object')

In [105]:
import datetime
import warnings
import pickle
from dataclasses import dataclass
from datetime import date
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



warnings.filterwarnings(
    action="ignore",
    message=".*Could not infer format.*",
)

warnings.filterwarnings(
    action="ignore",
    message=".*autologging encountered a warning.*",
)

warnings.filterwarnings(
    action="ignore",
    message=".*No visible GPU is found.*",
)

def load_numeric(file, datetime_to_numeric=True):
    csv = pd.read_csv(file)

    csv['created_date'] = pd.to_datetime(csv['created_date'])
    csv['refitted_date'] = pd.to_datetime(csv['refitted_date'])
    csv['refitted_date'] = csv['refitted_date'].fillna(csv['created_date'])
    
    csv['sale_date'] = pd.to_datetime(csv['sale_date'])
    age_baseline = max(csv["created_date"].max(), csv["refitted_date"].max(), csv["sale_date"].max())

    calculate_age = lambda x: age_baseline.year - x.year - ((age_baseline.month, age_baseline.day) < (x.month, x.day))
    csv['created_age']  = csv['created_date'].apply(calculate_age)
    csv['refitted_age']  = csv['refitted_date'].apply(calculate_age)
    csv['sale_age']  = csv['sale_date'].apply(calculate_age)
                                                                                                                                      
    if datetime_to_numeric:
        csv['created_date'] = pd.to_numeric(csv['created_date'])
        csv['refitted_date'] = pd.to_numeric(csv['refitted_date'])
        csv['sale_date'] = pd.to_numeric(csv['sale_date'])
    print(csv.select_dtypes(include=['object']).dtypes)
    return csv

def load_train(datetime_to_numeric=True):
    return load_numeric(train_path, datetime_to_numeric=datetime_to_numeric)
def load_test(datetime_to_numeric=True):
    return load_numeric(test_path, datetime_to_numeric=datetime_to_numeric)

def store_submission(alias, test_data, model=None):
    now = datetime.datetime.now().replace(microsecond=0).isoformat().replace(":", "")
    filename = f"{output_path}/{now}_{alias}_{dataset_name}_submission"
    if 'id' not in test_data.columns:
        test_data['id'] = pd.read_csv('test.csv')['id']
    test_data[['id', 'price']].to_csv(f'{filename}.csv', index=False)

    if model is not None:
        model_path = f"{filename}.pkl"
        pickle.dump(model, open(model_path, "wb"))
    
def predict_and_store(alias, model, test_x):
    test_data = load_test()
    test_data['price'] = model.predict(test_x)
    store_submission(alias, test_data, model)
    return test_data


@dataclass
class LocalDataset:
    X: pd.DataFrame
    y: pd.DataFrame
    X_train: pd.DataFrame
    X_test: pd.DataFrame
    y_train: pd.DataFrame
    y_test: pd.DataFrame
    test_data: pd.DataFrame
    train_data: pd.DataFrame
    features: list

@dataclass
class ModelTrain:
    X = None
    y = None
    X_train = None
    X_test = None
    y_train  = None
    y_test = None 
    features = None
    model = None


def load_split(datetime_to_numeric=True, test_size=0.2, features=None, drop=None, standard_scaler=True):
    train_data = load_train(datetime_to_numeric=datetime_to_numeric)
    test_data = load_test(datetime_to_numeric=datetime_to_numeric)

    df = train_data
    X = df.drop(['price'], axis=1)
    if drop is not None:
        X = df.drop(drop, axis=1)
        test_data = test_data.drop(drop, axis=1)
    if features is None:
        features = list(X.columns)
    else:
        X = X[features]
        test_data = test_data[features]
        print(f"Keeping features {features}") 
    y = df['price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)

    X_train = pd.DataFrame(X_train, columns=X.columns)
    X_test = pd.DataFrame(X_test, columns=X.columns)
    test_data = pd.DataFrame(test_data, columns=X.columns)
    train_data = pd.DataFrame(train_data[X.columns], columns=X.columns)
    if standard_scaler:
        scaler = StandardScaler()
        X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
        test_data = pd.DataFrame(scaler.transform(test_data), columns=X.columns)
        train_data = pd.DataFrame(scaler.transform(train_data[X.columns]), columns=X.columns)
    train_data["price"] = y
    
    return LocalDataset(X, y, X_train, X_test, y_train, y_test, test_data, train_data, features)

def calculate_metrics(ld: LocalDataset, model, method, variant="", y_pred=None):
    if y_pred is None:
        y_pred = model.predict(ld.X_test)
    print('MSE: ', mean_squared_error(ld.y_test, y_pred))
    print('RMSE: ', np.sqrt(mean_squared_error(ld.y_test, y_pred)))
    print('MAE: ', mean_absolute_error(ld.y_test, y_pred))
    metrics = {'rmse':  np.sqrt(mean_squared_error(ld.y_test, y_pred)),
               'mse': mean_squared_error(ld.y_test, y_pred), 
               'mae': mean_absolute_error(ld.y_test, y_pred)}
    results.loc[len(results)+1] = {'dataset': dataset_name, 
                         'method': method, 
                         'variant': variant, 
                         'rmse':  np.sqrt(mean_squared_error(ld.y_test, y_pred)),
                         'mse': mean_squared_error(ld.y_test, y_pred), 
                         'mae': mean_absolute_error(ld.y_test, y_pred)}
    mlflow.log_metrics(metrics)
    return results.sort_values(by='rmse')

_ = load_split()

Series([], dtype: object)
Series([], dtype: object)


# Sample code

In [47]:
def execute_sample_code(features=['cabins', 'decks', 'bathrooms', 'radar'], variant=''):
    import numpy as np
    import pandas as pd
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error

    with mlflow.start_run(
    run_name=f"linear-{variant}",
    tags={"version": "v1", "library": "sklearn", "optimization": "n/a"},
    description="sklearn",
):
        # load training and test data
        ld = load_split(features=features)
        train_data = ld.X_train
        # choose 4 features and price as training target
        # features = ['cabins', 'decks', 'bathrooms', 'radar']
        # train_x = train_data[features]
        # train_y = ld.y_train
    
        # simple linear regression model
        # price = a * cabins + b * decs + c * bathrooms + d * radar + e
        model = LinearRegression().fit(ld.X_train, ld.y_train)
    
        # predict price on test data
        # test_x = ld.test_data[features]
        # test_data = predict_and_store('linear_regression', model, test_x)
    
        # y_pred = model.predict(ld.X_test)
        # print('MSE: ', mean_squared_error(ld.y_test, y_pred))
        # print('RMSE: ', np.sqrt(mean_squared_error(ld.y_test, y_pred)))
        # print('MAE: ', np.sqrt(mean_absolute_error(ld.y_test, y_pred)))
        calculate_metrics(ld, model, method='linear', variant=variant)

        # mlflow.sklearn.log_model(sk_model=model, input_example=ld.X_train.sample(10))
        # mlflow.log_params(params)
    
        
        return model, ld.X_train
  
sample_model, sample_x = None, None
sample_model, sample_x = execute_sample_code(variant='demo')

Series([], dtype: object)


2024/05/23 19:12:06 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}


Series([], dtype: object)
Keeping features ['cabins', 'decks', 'bathrooms', 'radar']
MSE:  780428937023.6322
RMSE:  883418.8910271459
MAE:  610430.3268213461


# SelectKBest features

* https://lifewithdata.com/2022/03/19/feature-selection-with-selectkbest-in-scikit-learn/

In [48]:
def select_features(k=10):
    from sklearn.feature_selection import SelectKBest, chi2
    from sklearn.preprocessing import MinMaxScaler

    # Assuming X and y are your input features and target labels, respectively

    # Select the top 10 features
    selector = SelectKBest(score_func=chi2, k=k)

    df = load_train()
    X = df[[col for col in df.columns if col != 'price']]
    y = df['price']

    sc = MinMaxScaler()
    X_sc = sc.fit_transform(X)

    # X_new contains the selected features
    X_new = selector.fit_transform(X_sc, y)
    X_new

    kept_features = pd.DataFrame({'columns': X.columns,
                                  'Kept': selector.get_support()})
#     print(kept_features)

    new_train_csv = X.iloc[:,selector.get_support()]
    print(list(new_train_csv.columns))
    return list(new_train_csv.columns), new_train_csv
    
kept_features, train_csv_kept_features = select_features()
train_csv_kept_features

Series([], dtype: object)
['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


,decks,maintenance_area,weight_distribution_y,radar,bow_thruster,autopilot,solar_panels,navi_quality,refitted_age,sale_age
0,1.0,50.0,0.016326,0,1,0,0,0,17,1
1,1.0,38.0,0.105376,0,1,1,0,0,8,1
2,2.0,0.0,0.092526,0,0,0,0,0,9,1
3,1.0,47.5,-0.005674,0,1,1,0,2,15,1
4,1.5,66.5,0.149176,0,1,1,0,0,11,1
...,...,...,...,...,...,...,...,...,...,...
15124,1.0,0.0,0.116376,0,0,1,0,0,9,0
15125,2.0,0.0,0.079526,0,1,1,0,0,8,0
15126,1.0,30.0,-0.062674,0,0,0,0,0,13,0
15127,1.0,33.5,-0.065674,0,0,0,0,0,18,0


In [70]:
_ = execute_sample_code(features=kept_features, variant='kbest10')

Series([], dtype: object)
Series([], dtype: object)
Keeping features ['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


2024/05/23 19:19:32 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}


MSE:  682649835361.2692
RMSE:  826226.2616022739
MAE:  524141.92405948415


In [71]:
_ = execute_sample_code(features=select_features(k=5)[0], variant='kbest5')

Series([], dtype: object)
['radar', 'bow_thruster', 'solar_panels', 'navi_quality', 'sale_age']
Series([], dtype: object)
Series([], dtype: object)
Keeping features ['radar', 'bow_thruster', 'solar_panels', 'navi_quality', 'sale_age']


2024/05/23 19:19:34 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}


MSE:  934686115865.038
RMSE:  966791.6610444247
MAE:  662190.5646641604


In [76]:
_ = execute_sample_code(features=select_features(k=15)[0], variant='kbest15')

Series([], dtype: object)
['id', 'sale_date', 'decks', 'producer', 'cabins_area', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'created_age', 'refitted_age', 'sale_age']
Series([], dtype: object)
Series([], dtype: object)
Keeping features ['id', 'sale_date', 'decks', 'producer', 'cabins_area', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'created_age', 'refitted_age', 'sale_age']


2024/05/23 19:21:37 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}


MSE:  386139113370.5773
RMSE:  621400.9280412906
MAE:  409687.4073583644


In [74]:
_ = execute_sample_code(features=select_features(k=20)[0], variant='kbest20')

Series([], dtype: object)
['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_date', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'overall_quality', 'created_age', 'refitted_age', 'sale_age']
Series([], dtype: object)
Series([], dtype: object)
Keeping features ['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_date', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'overall_quality', 'created_age', 'refitted_age', 'sale_age']


2024/05/23 19:20:45 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}


MSE:  330157003585.2916
RMSE:  574592.9024842646
MAE:  381001.8122813862


# SkLearn Pipeline

https://machinelearningmastery.com/feature-extraction-on-tabular-data/

In [51]:
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

train_data = load_train()
test_data = load_test()

df = train_data
X = df[[col for col in df.columns if col != 'price']]
# features = list(new_train_csv.columns)
# X = df[features]
y = df['price']

# X = X.astype('float')
# y = LabelEncoder().fit_transform(y.astype('str'))

print(X.shape, y.shape)


Series([], dtype: object)
Series([], dtype: object)
(15129, 27) (15129,)


In [52]:
model = LogisticRegression(solver='liblinear')
# cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
# scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [110]:
results.sort_values(by='rmse')

,dataset,method,variant,rmse,mse,mae
22,original,autogluon,medium_quality,1.799401e+05,3.237845e+10,1.267031e+05
23,original,autogluon,medium_quality,1.799401e+05,3.237845e+10,1.267031e+05
26,original,xgboost,,3.771353e+05,1.422310e+11,2.146385e+05
24,original,xgboost,,3.771353e+05,1.422310e+11,2.146385e+05
29,original,xgboost,default,3.771353e+05,1.422310e+11,2.146385e+05
28,original,xgboost,kbest20,4.355712e+05,1.897223e+11,2.274063e+05
25,original,xgboost,kbest20,4.355712e+05,1.897223e+11,2.274063e+05
20,original,linear,kbest20,5.745929e+05,3.301570e+11,3.810018e+05
19,original,linear,kbest15,6.214009e+05,3.861391e+11,4.096874e+05
21,original,linear,kbest15,6.214009e+05,3.861391e+11,4.096874e+05


In [54]:
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

# test_data = predict_and_store('linear_regression_pipeline', model, X)
# mean_absolute_error(train_data['price'], model.predict(X))

In [55]:
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

# transforms for the feature union
transforms = list()
transforms.append(('mms', MinMaxScaler()))
transforms.append(('ss', StandardScaler()))
transforms.append(('rs', RobustScaler()))
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))
# create the feature union
fu = FeatureUnion(transforms)

In [56]:
model = LogisticRegression(solver='liblinear')
rfe = RFE(estimator=LogisticRegression(solver='liblinear'), n_features_to_select=8)
steps = list()
steps.append(('fu', fu))
steps.append(('rfe', rfe))
steps.append(('m', model))
pipeline = Pipeline(steps=steps)

# cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)
# scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, verbose=1)
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Autogluon
* https://auto.gluon.ai/stable/tutorials/tabular/tabular-feature-engineering.html
* https://auto.gluon.ai/stable/tutorials/tabular/advanced/index.html

In [57]:
!pip install -U autogluon ipywidgets
# !pip uninstall lightgbm -y
# !pip install lightgbm --install-option=--gpu

In [58]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

ld_autogluon = load_split(datetime_to_numeric=False, standard_scaler=False)

X = ld_autogluon.X
y = ld_autogluon.y

Series([], dtype: object)
Series([], dtype: object)


In [59]:
label = 'price'
ld_autogluon.train_data[label].describe()

count    1.512900e+04
mean     1.615167e+06
std      1.077826e+06
min      2.250000e+05
25%      9.630000e+05
50%      1.350000e+06
75%      1.923000e+06
max      2.310000e+07
Name: price, dtype: float64

In [60]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
auto_ml_pipeline_feature_generator.fit_transform(X=X, y=y)

,id,cabins,decks,bathrooms,producer,total_area,crew_area,cabins_area,maintenance_area,crew_area_coef,...,sale_date.day,sale_date.dayofweek,created_date,created_date.year,created_date.month,created_date.dayofweek,refitted_date,refitted_date.year,refitted_date.month,refitted_date.dayofweek
0,1,3,1.0,1,59,364.6,118.0,68.0,50.0,93.0,...,28,5,4310064000000000000,2106,8,6,4310064000000000000,2106,8,6
1,2,3,1.0,3,24,998.3,92.5,54.5,38.0,70.5,...,28,5,4594060800000000000,2115,8,3,4594060800000000000,2115,8,3
2,3,4,2.0,2,24,1793.4,135.5,135.5,0.0,139.0,...,28,5,4573065600000000000,2114,12,5,4573065600000000000,2114,12,5
3,4,3,1.0,2,51,327.5,119.5,72.0,47.5,100.5,...,28,5,4354819200000000000,2108,1,6,4354819200000000000,2108,1,6
4,5,4,1.5,2,41,2144.2,181.5,115.0,66.5,141.5,...,28,5,4507401600000000000,2112,11,1,4507401600000000000,2112,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,15125,3,1.0,2,14,2243.3,70.5,70.5,0.0,147.5,...,10,4,4544208000000000000,2114,1,0,4544208000000000000,2114,1,0
15125,15126,4,2.0,2,20,364.5,101.0,101.0,0.0,104.5,...,10,4,4602009600000000000,2115,11,4,4602009600000000000,2115,11,4
15126,15127,3,1.0,1,43,200.0,95.0,65.0,30.0,94.0,...,10,4,4417977600000000000,2110,1,2,4417977600000000000,2110,1,2
15127,15128,2,1.0,1,50,333.5,69.5,36.0,33.5,46.0,...,20,0,4270579200000000000,2105,5,4,4270579200000000000,2105,5,4


In [69]:
with mlflow.start_run(
    run_name="linear-automlfeatures",
    tags={"version": "v1", "library": "sklearn", "optimization": "AutoMLPipelineFeatureGenerator"},
    description="sklearn with AutoMLPipelineFeatureGenerator",
):
    model = LinearRegression().fit(auto_ml_pipeline_feature_generator.transform(X=ld_autogluon.X_train), 
                                   ld_autogluon.y_train)
    calculate_metrics(ld_autogluon, model, method='linear', 
                      variant='automlfeatures',
                      y_pred=model.predict(auto_ml_pipeline_feature_generator.transform(X=ld_autogluon.X_test)))

2024/05/23 19:17:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2024/05/23 19:17:35 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during

MSE:  493223782573.8531
RMSE:  702298.9267924685
MAE:  481399.6393533197


In [78]:
time_limit = 3600
presets='medium_quality'

predictor = None
with mlflow.start_run(
    run_name="autogluon",
    tags={"version": "v1", "library": "autogluon", "optimization": presets},
    description="autogluon",
):
    predictor = TabularPredictor(label=label, problem_type='regression').fit(train_data, num_gpus=1, presets=presets,
                                                  time_limit=time_limit, dynamic_stacking=False,
                                                  feature_generator=auto_ml_pipeline_feature_generator)
    calculate_metrics(ld_autogluon, model, method='autogluon', 
                      variant='medium_quality',
                      y_pred=predictor.predict(auto_ml_pipeline_feature_generator.transform(X=ld_autogluon.X_test)))
    metrics = predictor.evaluate(ld_autogluon.train_data, silent=True)
    mlflow.log_metrics(metrics)
    predictor.leaderboard(ld_autogluon.train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240523_222704"
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20240523_222704"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.4
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #40~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 16 10:53:04 UTC 2
CPU Count:          8
Memory Avail:       48.35 GB / 62.58 GB (77.3%)
Disk Space Avail:   136.89 GB / 848.70 GB (16.1%)
Train Data Rows:    15129
Train Data Columns: 27
Label Column:       price
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
AutoMLPipelineFeatureGenerator is already fit, so the training data will be processed via .transform() instead of .fit_transform().
	Types of features in original data (raw dtype, special dtypes):
		('datetime', []) : 

[1000]	valid_set's rmse: 323937


2024/05/23 19:27:28 ERROR mlflow.lightgbm: Failed to log feature importance plot. LightGBM autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/lightgbm/__init__.py", line 828, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/lightgbm/__init__.py", line 714, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/tracking/fluent.py", line 1068, in log_artifact
    MlflowClient().log_artifact(run_id, local_path, artifact_path)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/trac

MSE:  32378448349.640022
RMSE:  179940.12434596132
MAE:  126703.11779163913


In [79]:
predictor.evaluate(train_data, silent=True)

{'root_mean_squared_error': -184353.69722113048,
 'mean_squared_error': -33986285679.100254,
 'mean_absolute_error': -124712.11734086853,
 'r2': 0.9707426369568309,
 'pearsonr': 0.9854215176193719,
 'median_absolute_error': -87046.8125}

In [80]:
predictor.leaderboard(train_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,-123021.035214,-3.231885e+05,root_mean_squared_error,0.534749,0.090500,13.182128,0.534749,0.090500,13.182128,1,True,11
1,RandomForestMSE,-181516.840704,-3.603472e+05,root_mean_squared_error,1.197038,0.235468,25.934293,1.197038,0.235468,25.934293,1,True,5
2,LightGBM,-181900.755922,-3.156915e+05,root_mean_squared_error,0.232341,0.069997,5.974942,0.232341,0.069997,5.974942,1,True,4
3,WeightedEnsemble_L2,-184353.697221,-2.968269e+05,root_mean_squared_error,1.331720,0.239762,67.523276,0.006761,0.000660,0.025834,2,True,12
4,ExtraTreesMSE,-185543.899567,-3.573955e+05,root_mean_squared_error,0.773379,0.252939,10.240706,0.773379,0.252939,10.240706,1,True,7
5,XGBoost,-188157.518668,-3.325779e+05,root_mean_squared_error,0.179162,0.029225,22.928612,0.179162,0.029225,22.928612,1,True,9
6,CatBoost,-189875.871289,-3.052288e+05,root_mean_squared_error,0.033250,0.005646,14.382585,0.033250,0.005646,14.382585,1,True,6
7,LightGBMXT,-199697.985315,-3.235772e+05,root_mean_squared_error,0.539832,0.057525,7.174850,0.539832,0.057525,7.174850,1,True,3
8,NeuralNetFastAI,-267834.016880,-3.330556e+05,root_mean_squared_error,0.524619,0.072959,33.957787,0.524619,0.072959,33.957787,1,True,8
9,NeuralNetTorch,-299473.606573,-3.864554e+05,root_mean_squared_error,0.169648,0.052473,75.516818,0.169648,0.052473,75.516818,1,True,10


In [82]:
test_data = load_test(datetime_to_numeric=False)
df = auto_ml_pipeline_feature_generator.transform(test_data)
test_x = df.drop([], axis=1)
predict_and_store(f'autogluon_{presets}', predictor, test_x)

Series([], dtype: object)
Series([], dtype: object)


,id,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,crew_area,...,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality,created_age,refitted_age,sale_age,price
0,15130,4825267200000000000,4,2.0,3,41,4620412800000000000,4620412800000000000,361.6,155.5,...,1,1,0,0,3,8,7,7,1,1136958.125
1,15131,4825267200000000000,3,1.0,2,3,4559846400000000000,4559846400000000000,541.7,104.5,...,0,0,0,0,4,8,9,9,1,909334.750
2,15132,4825267200000000000,4,2.0,2,37,4665168000000000000,4665168000000000000,167.2,143.0,...,1,1,0,0,3,8,6,6,1,1664548.125
3,15133,4825267200000000000,3,1.0,2,52,4328380800000000000,4328380800000000000,237.5,130.0,...,0,0,0,0,4,9,16,16,1,2934944.500
4,15134,4825267200000000000,4,2.0,2,29,4644000000000000000,4644000000000000000,2149.9,165.5,...,0,1,0,0,3,9,6,6,1,2648848.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6479,21609,4857753600000000000,4,2.0,3,36,4659897600000000000,4659897600000000000,902.1,124.5,...,1,1,0,0,3,8,6,6,0,1545018.125
6480,21610,4857840000000000000,4,1.0,1,51,4389120000000000000,4389120000000000000,292.5,99.0,...,1,0,0,1,4,8,14,14,0,1939362.125
6481,21611,4857840000000000000,5,2.0,3,43,4665168000000000000,4665168000000000000,267.7,166.0,...,1,1,0,0,3,9,6,6,0,3982977.500
6482,21612,4857840000000000000,4,2.0,2,24,4659897600000000000,4659897600000000000,589.0,129.0,...,0,0,0,0,3,9,6,6,0,1360106.375


# XGBoost

* https://github.com/dmlc/xgboost/blob/master/demo/guide-python/sklearn_examples.py

In [14]:
!pip install xgboost

# bst = xgboost.train(...)
# config = bst.save_config()
# print(config)

In [97]:
from sklearn.datasets import make_classification

def execute_xgboost(features=None, variant='default'):
    from numpy import mean
    from numpy import std
    import numpy as np
    import xgboost as xgb

    with mlflow.start_run(
    run_name=f"xgboost-{variant}",
    tags={"version": "v1", "library": "xgboost", "optimization": "n/a"},
    description="xgboost",
):
        ld = load_split(features=features)
        train_data = ld.X_train

        model = xgb.XGBRegressor(n_jobs=1, tree_method="hist", device="cuda")
        model = model.fit(ld.X_train, ld.y_train)
    
        calculate_metrics(ld, model, method='xgboost', variant=variant)
        
        return model, ld.X_train

In [96]:
_ = execute_xgboost(features=select_features(k=10)[0], variant='kbest10')
_ = execute_xgboost(features=select_features(k=20)[0], variant='kbest20')
_ = execute_xgboost()


Series([], dtype: object)
['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']
Series([], dtype: object)
Series([], dtype: object)
Keeping features ['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:53:00] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
2024/05/23 19:53:01 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 684, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 583, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/pyt

MSE:  511535271068.4332
RMSE:  715216.939864006
MAE:  401716.0079209352
Series([], dtype: object)
['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_date', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'overall_quality', 'created_age', 'refitted_age', 'sale_age']
Series([], dtype: object)
Series([], dtype: object)
Keeping features ['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_date', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'overall_quality', 'created_age', 'refitted_age', 'sale_age']


/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:53:11] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
2024/05/23 19:53:12 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 684, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 583, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/pyt

MSE:  189722287527.2487
RMSE:  435571.21981054795
MAE:  227406.27618968935
Series([], dtype: object)
Series([], dtype: object)


/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:53:23] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
2024/05/23 19:53:24 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 684, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 583, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/pyt

MSE:  142231004580.83203
RMSE:  377135.26032556547
MAE:  214638.48439565432


In [108]:

def execute_xgboost_cv(features=None, variant='default'):
    from numpy import mean
    from numpy import std
    import numpy as np
    import xgboost as xgb
    from sklearn.model_selection import RepeatedKFold
    from sklearn.model_selection import cross_val_score


    with mlflow.start_run(
    run_name=f"xgboost-cv-{variant}",
    tags={"version": "v1", "library": "xgboost", "optimization": "cv"},
    description="xgboost cross validation",
):
        ld = load_split(features=features)
        train_data = ld.X_train

        model = xgb.XGBRegressor(n_jobs=1, tree_method="hist", device="cuda")
        model = model.fit(ld.X, ld.y)

        cv = RepeatedKFold(n_splits=3, n_repeats=10, random_state=1)
        scores = cross_val_score(model, ld.X, ld.y, cv=cv, n_jobs=-1, verbose=1)

        calculate_metrics(ld, model, method='xgboost-cv', variant=variant)
        mlflow.log_metrics({'cv-mean': mean(scores), 'cv-std': std(scores)})
        
        return model, ld.X


# xgb_model = xgb.XGBRegressor(n_jobs=1, tree_method="hist", device="cuda")
# xgb_model.fit(X, y)
# cv = RepeatedKFold(n_splits=3, n_repeats=10, random_state=1)
# scores = cross_val_score(xgb_model, X, y, cv=cv, n_jobs=-1, verbose=1)
# print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [109]:
_ = execute_xgboost_cv(features=select_features(k=10)[0], variant='kbest10')
_ = execute_xgboost_cv(features=select_features(k=20)[0], variant='kbest20')
_ = execute_xgboost_cv()


Series([], dtype: object)
['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']
Series([], dtype: object)
Series([], dtype: object)
Keeping features ['decks', 'maintenance_area', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'refitted_age', 'sale_age']


2024/05/23 20:05:35 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 684, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 583, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/tracking/fluent.py", line 1068, in log_artifact
    MlflowClient().log_artifact(run_id, local_path, artifact_path)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/tracking

MSE:  1099763916020.3745
RMSE:  1048696.29350941
MAE:  635314.5038830139
Series([], dtype: object)
['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_date', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'overall_quality', 'created_age', 'refitted_age', 'sale_age']
Series([], dtype: object)
Series([], dtype: object)
Keeping features ['id', 'sale_date', 'decks', 'producer', 'created_date', 'refitted_date', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'weight_distribution_y', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'overall_quality', 'created_age', 'refitted_age', 'sale_age']


2024/05/23 20:05:38 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 684, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 583, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/tracking/fluent.py", line 1068, in log_artifact
    MlflowClient().log_artifact(run_id, local_path, artifact_path)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/tracking

MSE:  2763699753510.8174
RMSE:  1662437.894632704
MAE:  1344822.7298401645
Series([], dtype: object)
Series([], dtype: object)


2024/05/23 20:05:42 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 684, in train_impl
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/xgboost/__init__.py", line 583, in log_feature_importance_plot
    mlflow.log_artifact(filepath)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/tracking/fluent.py", line 1068, in log_artifact
    MlflowClient().log_artifact(run_id, local_path, artifact_path)
  File "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/tracking

MSE:  2623506765520.74
RMSE:  1619724.2868836476
MAE:  1279643.6296598103


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    2.7s finished


In [22]:
test_data = load_test()
df = test_data
test_x = df[[col for col in df.columns if col != 'price']]
test_data = predict_and_store('xgboost', xgb_model, test_x)

Series([], dtype: object)
Series([], dtype: object)


# Optuna

* https://optuna.org/
* https://practicaldatascience.co.uk/machine-learning/how-to-tune-an-xgbregressor-model-with-optuna
* https://practicaldatascience.co.uk/machine-learning/how-to-use-optuna-for-xgboost-hyperparameter-tuning
* https://medium.com/optuna/using-optuna-to-optimize-xgboost-hyperparameters-63bfcdfd3407
* https://randomrealizations.com/posts/xgboost-parameter-tuning-with-optuna/

In [44]:
! pip install optuna --quiet

In [145]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import optuna
from wandb.integration.xgboost import WandbCallback

[2024-05-15 17:14:58 -0300] [2594208] [CRITICAL] WORKER TIMEOUT (pid:2594237)
[2024-05-15 17:14:58 -0300] [2594208] [CRITICAL] WORKER TIMEOUT (pid:2594239)
[2024-05-15 17:14:58 -0300] [2594208] [CRITICAL] WORKER TIMEOUT (pid:2594247)
[2024-05-15 17:14:58 -0300] [2594208] [CRITICAL] WORKER TIMEOUT (pid:2594248)
[2024-05-15 17:14:58 -0300] [2594239] [INFO] Worker exiting (pid: 2594239)
[2024-05-15 17:14:58 -0300] [2594237] [INFO] Worker exiting (pid: 2594237)
[2024-05-15 17:14:58 -0300] [2594247] [INFO] Worker exiting (pid: 2594247)
[2024-05-15 17:14:58 -0300] [2594248] [INFO] Worker exiting (pid: 2594248)
[2024-05-15 17:14:59 -0300] [2594208] [ERROR] Worker (pid:2594237) exited with code 1
[2024-05-15 17:14:59 -0300] [2594208] [ERROR] Worker (pid:2594237) exited with code 1.
[2024-05-15 17:14:59 -0300] [2594208] [ERROR] Worker (pid:2594239) exited with code 1
[2024-05-15 17:14:59 -0300] [2594208] [ERROR] Worker (pid:2594239) exited with code 1.
[2024-05-15 17:14:59 -0300] [3324216] [INF

In [115]:
def set_experiment(name):
    mlflow.set_experiment(name)
    
# run = wandb.init()

set_experiment("Default")

xgboost_optuna_baseline = None
with mlflow.start_run(
    run_name="xgboost_optuna_baseline",
    tags={"version": "v1", "library": "xgboost", "optimization": "none"},
    description="xgboost_optuna_baseline",
):
    train_data = load_train()
    test_data = load_test()
    
    df = train_data
    X = df.drop(['price'], axis=1)
    y = df['price']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    
    model = xgb.XGBRegressor(n_jobs=-1, tree_method="hist", device="cuda", 
                             # callbacks=[WandbCallback(log_model=True)]
                            )
    model.fit(X_train, y_train)
    
    # run.finish()
    
    y_pred = model.predict(X_test)
    xgboost_optuna_baseline = model
    
    print('MSE: ', mean_squared_error(y_test, y_pred))
    print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

Series([], dtype: object)
Series([], dtype: object)


/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:15:36] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
2024/05/15 13:15:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:15:37] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


MSE:  156406649734.38065
RMSE:  395482.80586440244


In [137]:
set_experiment("default")
xgboost_study = None
xgboost_optuna_model = None
with mlflow.start_run(
    run_name="xgboost_optuna",
    tags={"version": "v1", "library": "xgboost", "optimization": "optuna"},
    description="xgboost_optuna",
) as parent_run:
    mlflow.log_param("parent", "yes")

    def objective(trial):
        with mlflow.start_run(
            run_name=f"xgboost_optuna_trial_{run_number}",
            description="xgboost_optuna_trial",
            tags={"version": "v1", "library": "xgboost", "optimization": "optuna", "trial": "true"},
            nested=True,
        ):
            param = {
                'max_depth': trial.suggest_int('max_depth', 1, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
                'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'gamma': trial.suggest_float('gamma', 0.01, 1.0),
                'subsample': trial.suggest_float('subsample', 0.01, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
                'random_state': trial.suggest_int('random_state', 1, 1000),
                'device': 'cuda',
                'tree_method': 'hist',
            }
            model = xgboost.XGBRegressor(**param)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            return mean_squared_error(y_test, y_pred)
    
    study = optuna.create_study(direction='minimize', study_name='regression')
    study.optimize(objective, n_trials=10)

    model = xgboost.XGBRegressor(**study.best_params)
    model.fit(X, y)
    y_pred = model.predict(X_test)
    xgboost_study = study
    xgboost_optuna_model = model

    
    print('MSE: ', mean_squared_error(y_test, y_pred))
    print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

[I 2024-05-15 14:46:47,220] A new study created in memory with name: regression
/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:46:47] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
2024/05/15 14:46:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:46:49] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
[I 2024-05-15 14:46:52,704] Trial 0 finished with value: 202381755399.57178 and parameters: {'max_depth': 10, 'learning_rate': 0.08393809455611914, 'n_estimators': 424, 'min_child_weight': 1, 'gamm

MSE:  1853613621270.2583
RMSE:  1361474.7964138954


In [55]:
print('Best parameters', xgboost_study.best_params)
print('Best value', xgboost_study.best_value)
print('Best trial', xgboost_study.best_trial)

Best parameters {'max_depth': 8, 'learning_rate': 0.04216722893316317, 'n_estimators': 899, 'min_child_weight': 8, 'gamma': 0.44670517525549064, 'subsample': 0.5320770493865845, 'colsample_bytree': 0.6761495876719824, 'reg_alpha': 0.7713228879524867, 'reg_lambda': 0.9288029580977616, 'random_state': 886}
Best value 135940135043.7962
Best trial FrozenTrial(number=64, state=1, values=[135940135043.7962], datetime_start=datetime.datetime(2024, 5, 15, 11, 23, 9, 653573), datetime_complete=datetime.datetime(2024, 5, 15, 11, 23, 12, 23897), params={'max_depth': 8, 'learning_rate': 0.04216722893316317, 'n_estimators': 899, 'min_child_weight': 8, 'gamma': 0.44670517525549064, 'subsample': 0.5320770493865845, 'colsample_bytree': 0.6761495876719824, 'reg_alpha': 0.7713228879524867, 'reg_lambda': 0.9288029580977616, 'random_state': 886}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=10, log=False, low=1, step=1), 'learning_rate': FloatDis

In [56]:
xgboost_optuna_model = xgboost.XGBRegressor(**xgboost_study.best_params)
xgboost_optuna_model.fit(X, y)
y_pred = xgboost_optuna_model.predict(X_test)

print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

2024/05/15 11:36:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0c26b1979c024b6c829ec37ba3cb81c9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2024/05/15 11:36:46 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have m

MSE:  2042877622723.3774
RMSE:  1429292.7001574512


In [57]:
test_data = load_test()
df = test_data
test_x = df.drop(['id'], axis=1)
predict_and_store('xgboost_optuna', xgboost_optuna_model, test_x)


Series([], dtype: object)
Series([], dtype: object)


ValueError: feature_names mismatch: ['id', 'sale_date', 'cabins', 'decks', 'bathrooms', 'producer', 'created_date', 'refitted_date', 'total_area', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'total_area_coef', 'weight_distribution_x', 'weight_distribution_y', 'engine_thrust', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'state', 'overall_quality'] ['sale_date', 'cabins', 'decks', 'bathrooms', 'producer', 'created_date', 'refitted_date', 'total_area', 'crew_area', 'cabins_area', 'maintenance_area', 'crew_area_coef', 'total_area_coef', 'weight_distribution_x', 'weight_distribution_y', 'engine_thrust', 'radar', 'bow_thruster', 'autopilot', 'solar_panels', 'navi_quality', 'state', 'overall_quality']
expected id in input data

# Optuna + LightGBM
* https://practicaldatascience.co.uk/machine-learning/how-to-tune-a-lightgbmclassifier-model-with-optuna
* https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.htmloptuna

In [121]:
set_experiment("default")
import lightgbm as lgbm

dataset = load_split(datetime_to_numeric=True)

lightgbm_model = None
with mlflow.start_run(
    run_name="lightgbm_optuna_baseline",
    tags={"version": "v1", "library": "lightgbm", "optimization": "none"},
    description="lightgbm_optuna_baseline",
):
    model = lgbm.LGBMRegressor()
    model.fit(X_train, y_train)
    
    # run.finish()
    
    y_pred = model.predict(X_test)
    lightgbm_model = model
    
    print('MSE: ', mean_squared_error(y_test, y_pred))
    print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

Series([], dtype: object)
Series([], dtype: object)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3174
[LightGBM] [Info] Number of data points in the train set: 10590, number of used features: 24
[LightGBM] [Info] Start training from score 1616687.228329
MSE:  149810864507.44727
RMSE:  387054.0847316396


In [131]:
set_experiment("default")

lightgbm_study = None
lightgbm_optuna_model = None
with mlflow.start_run(
    run_name="lightgbm_optuna",
    tags={"version": "v1", "library": "lightgbm", "optimization": "optuna"},
    description="lightgbm_optuna",
) as parent_run:
    mlflow.log_param("parent", "yes")

    def objective(trial):
        with mlflow.start_run(
            run_name=f"lightgbm_optuna_trial_{run_number}",
            description="lightgbm_optuna_trial",
            tags={"version": "v1", "library": "lightgbm", "optimization": "optuna", "trial": "true"},
            nested=True,
        ):
            param = {
                "learning_rate": trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
                "max_depth": trial.suggest_int("max_depth", 1, 9),
                "n_estimators": trial.suggest_categorical("n_estimators", [7000, 15000, 20000]),
                'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                'metric': 'rmse',
                'verbose': -1,
            }            
            model = lgbm.LGBMRegressor(**param)
            model.fit(X_train, y_train, eval_metric='rmse')
            y_pred = model.predict(X_test)
            print('MSE: ', mean_squared_error(y_test, y_pred))
            print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))
            return mean_squared_error(y_test, y_pred)
    
    study = optuna.create_study(direction='minimize', study_name='lightgbm_regression')
    study.optimize(objective, n_trials=100)

    model = lgbm.LGBMRegressor(**study.best_params)
    model.fit(X, y)
    y_pred = model.predict(X_test)
    lightgbm_study = study
    lightgbm_optuna_model = model
    
    
    print('MSE: ', mean_squared_error(y_test, y_pred))
    print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

[I 2024-05-15 13:58:21,973] A new study created in memory with name: lightgbm_regression
[I 2024-05-15 13:58:41,041] Trial 0 finished with value: 190583272521.3278 and parameters: {'learning_rate': 0.04252661490210386, 'max_depth': 9, 'n_estimators': 20000, 'lambda_l1': 0.0028743778285412022, 'lambda_l2': 0.07934595610539948, 'num_leaves': 167, 'feature_fraction': 0.8048440718913019, 'bagging_fraction': 0.7100716121139077, 'bagging_freq': 5, 'min_child_samples': 85}. Best is trial 0 with value: 190583272521.3278.
[I 2024-05-15 13:58:49,468] Trial 1 finished with value: 186946764404.54315 and parameters: {'learning_rate': 0.040454791614786684, 'max_depth': 5, 'n_estimators': 15000, 'lambda_l1': 0.2102014546306462, 'lambda_l2': 0.2075695391708165, 'num_leaves': 126, 'feature_fraction': 0.5446873228653816, 'bagging_fraction': 0.43486380274171704, 'bagging_freq': 4, 'min_child_samples': 95}. Best is trial 1 with value: 186946764404.54315.
[I 2024-05-15 13:58:54,620] Trial 2 finished with v

[LightGBM] [Warning] feature_fraction is set=0.9583384983209591, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9583384983209591
[LightGBM] [Warning] lambda_l2 is set=4.053773725636125, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.053773725636125
[LightGBM] [Warning] lambda_l1 is set=5.070476652713802, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.070476652713802
[LightGBM] [Warning] bagging_fraction is set=0.8378790738400035, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8378790738400035
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9583384983209591, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9583384983209591
[LightGBM] [Warning] lambda_l2 is set=4.053773725636125, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.053773725636125
[LightGBM] [Warning] lambda_l1 is set=5.07

2024/05/15 14:19:43 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


[LightGBM] [Warning] feature_fraction is set=0.9583384983209591, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9583384983209591
[LightGBM] [Warning] lambda_l2 is set=4.053773725636125, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.053773725636125
[LightGBM] [Warning] lambda_l1 is set=5.070476652713802, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.070476652713802
[LightGBM] [Warning] bagging_fraction is set=0.8378790738400035, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8378790738400035
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.9583384983209591, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9583384983209591
[LightGBM] [Warning] lambda_l2 is set=4.053773725636125, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.053773725636125
[LightGBM] [Warning] lambda_l1 is set=5.07

In [136]:
test_data = load_test()
df = test_data
test_x = df.drop([], axis=1)
predict_and_store('lightgbm_optuna', lightgbm_optuna_model, test_x)

Series([], dtype: object)
Series([], dtype: object)
[LightGBM] [Warning] feature_fraction is set=0.9583384983209591, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9583384983209591
[LightGBM] [Warning] lambda_l2 is set=4.053773725636125, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.053773725636125
[LightGBM] [Warning] lambda_l1 is set=5.070476652713802, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.070476652713802
[LightGBM] [Warning] bagging_fraction is set=0.8378790738400035, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8378790738400035
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


2024/05/15 14:43:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/gmatheu/.pyenv/versions/3.11.4/envs/intive-interstellar-sales-showdown/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


,id,sale_date,cabins,decks,bathrooms,producer,created_date,refitted_date,total_area,crew_area,...,weight_distribution_y,engine_thrust,radar,bow_thruster,autopilot,solar_panels,navi_quality,state,overall_quality,price
0,15130,4825267200000000000,4,2.0,3,41,4620412800000000000,-9223372036854775808,361.6,155.5,...,0.116076,1324.3,0,1,1,0,0,3,8,1.036359e+06
1,15131,4825267200000000000,3,1.0,2,3,4559846400000000000,-9223372036854775808,541.7,104.5,...,0.103176,2408.2,0,0,0,0,0,4,8,9.133864e+05
2,15132,4825267200000000000,4,2.0,2,37,4665168000000000000,-9223372036854775808,167.2,143.0,...,-0.106724,1111.5,0,1,1,0,0,3,8,1.628908e+06
3,15133,4825267200000000000,3,1.0,2,52,4328380800000000000,-9223372036854775808,237.5,130.0,...,-0.061624,1291.7,0,0,0,0,0,4,9,2.847699e+06
4,15134,4825267200000000000,4,2.0,2,29,4644000000000000000,-9223372036854775808,2149.9,165.5,...,-0.081574,2965.0,0,0,1,0,0,3,9,2.509291e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6479,21609,4857753600000000000,4,2.0,3,36,4659897600000000000,-9223372036854775808,902.1,124.5,...,0.071976,1528.8,0,1,1,0,0,3,8,1.663866e+06
6480,21610,4857840000000000000,4,1.0,1,51,4389120000000000000,-9223372036854775808,292.5,99.0,...,-0.000324,1597.0,0,1,0,0,1,4,8,2.082979e+06
6481,21611,4857840000000000000,5,2.0,3,43,4665168000000000000,-9223372036854775808,267.7,166.0,...,-0.047074,2173.0,0,1,1,0,0,3,9,4.034888e+06
6482,21612,4857840000000000000,4,2.0,2,24,4659897600000000000,-9223372036854775808,589.0,129.0,...,0.097126,2963.6,0,0,0,0,0,3,9,1.370282e+06


# cuML

* https://docs.rapids.ai/api/cuml/stable/cuml_intro/#where-possible-match-the-scikit-learn-api

# AutoXGB

* https://github.com/abhishekkrthakur/autoxgb

In [134]:
# !pip install autoxgb

In [132]:

# from autoxgb import AutoXGB


# # required parameters:
# train_filename = "train.csv"
# output = "autoxgb-output"

# # optional parameters
# test_filename = "test.csv"
# task = None
# idx = None
# targets = ["price"]
# features = None
# categorical_features = None
# use_gpu = False
# num_folds = 5
# seed = 42
# num_trials = 100
# time_limit = 360
# fast = True

# # Now its time to train the model!
# axgb = AutoXGB(
#     train_filename=train_filename,
#     output=output,
#     test_filename=test_filename,
#     task=task,
#     idx=idx,
#     targets=targets,
#     features=features,
#     categorical_features=categorical_features,
#     use_gpu=use_gpu,
#     num_folds=num_folds,
#     seed=seed,
#     num_trials=num_trials,
#     time_limit=time_limit,
#     fast=fast,
# )


In [133]:
# axgb.train()

# Sample

In [ ]:
sample_submission

,id,price
0,15130,1096000.0
1,15131,1580000.0
2,15132,1662000.0
3,15133,1735000.0
4,15134,1823000.0
...,...,...
6479,21609,1371000.0
6480,21610,1361000.0
6481,21611,1232000.0
6482,21612,1667000.0


In [38]:
! pip install -U pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2


# Notebooks with GPU
* https://www.kaggle.com/ (30 hours)
* https://colab.research.google.com/ (free-tier w GPU)
* https://www.paperspace.com/pricing (free-tier no GPU)
* https://saturncloud.io/plans/saturn_cloud_plans/ (free-tier w GPU?, Waitlist)
* https://deepnote.com/ (free-tier no GPU)
* https://studiolab.sagemaker.aws/login (wailist)


# MLFlow
* https://www.kaggle.com/code/sharanharsoor/mlflow-end-to-end-ml-models

In [38]:
!pip install mlflow 'mxnet<=1.9.1' --quiet 
!pip install pyngrok --quiet

In [11]:
import mlflow

get_ipython().system_raw("mlflow ui --port 5555 &")

In [12]:
import optuna
import xgboost
import lightgbm
import autogluon

import mlflow.gluon
import mlflow.lightgbm
import mlflow.xgboost 

mlflow.autolog()
mlflow.lightgbm.autolog()
mlflow.xgboost.autolog()
# mlflow.gluon.autolog()


2024/05/23 18:47:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/05/23 18:47:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2024/05/23 18:47:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
[2024-05-23 18:47:08 -0300] [1932392] [INFO] Starting gunicorn 22.0.0
[2024-05-23 18:47:08 -0300] [1932392] [INFO] Listening at: http://127.0.0.1:5555 (1932392)
[2024-05-23 18:47:08 -0300] [1932392] [INFO] Using worker: sync
[2024-05-23 18:47:08 -0300] [1932393] [INFO] Booting worker with pid: 1932393
[2024-05-23 18:47:08 -0300] [1932394] [INFO] Booting worker with pid: 1932394
[2024-05-23 18:47:09 -0300] [1932396] [INFO] Booting worker with pid: 1932396
[2024-05-23 18:47:09 -0300] [1932397] [INFO] Booting worker with pid: 1932397


# Weight and Biases
* https://colab.research.google.com/github/wandb/examples/blob/master/colabs/boosting/Credit_Scorecards_with_XGBoost_and_W%26B.ipynb#scrollTo=agAgfdIeuPdX

In [25]:

from pyngrok import ngrok
from getpass import getpass

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2VwE0ghPzEaPvlzIt9tdltGLFRZ_6nyxEMvYDUGjwfsW8A3xh"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5555", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://cc95-2803-9800-a880-82aa-9f1-c957-e5a5-cdf6.ngrok-free.app              


ERROR:pyngrok.process.ngrok:t=2024-05-15T10:28:04-0300 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=07377774eb87 clientid=a68eca9b639b12390393369d66eec66e
ERROR:pyngrok.process.ngrok:t=2024-05-15T10:28:04-0300 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=6172aedd9fb7 err="session closed"
ERROR:pyngrok.process.ngrok:t=2024-05-15T10:28:05-0300 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple tunnels on a single agent session using a configuration file.\nTo learn more, see https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config/\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n"


In [59]:
!pip install --upgrade -q wandb
import wandb

In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 



In [12]:
import wandb
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [24]:
wandb.login(key=wandb_api)

NameError: name 'wandb' is not defined

In [67]:
WANDB_PROJECT ='intive-interstellar-sales-showdown'
# run = wandb.init(project=WANDB_PROJECT, job_type='preprocess-data')
run = wandb.init(project=WANDB_PROJECT)



ds_art = wandb.Artifact(name='original', 
                                    type='dataset',
                                    description='Original dataset'
                                 )

# Attach our processed data to the Artifact 
ds_art.add_file(train_path)
ds_art.add_file(test_path)

table = wandb.Table(dataframe=load_train().sample(1000))
wandb.log({'dataset': table})


run.log_artifact(ds_art)   

run.finish()

Series([], dtype: object)
